In [100]:
import ipywidgets as wgt
import numpy as np
import pandas as pd
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.offline as py
#import plotly.figure_factory as ff
import plotly.graph_objs as go
tdf=pd.read_csv('chess2_rep_moreinfo.tab', sep='\t', index_col=0)
tdf['hasCDS']=np.where(tdf['cdlen'].ne(0),'CDS','NoCDS')
tdf[(tdf['hasCDS'].eq('NoCDS'))].head()

,gstatus,gtype,status,type,asm,chr,strand,excount,exlen,cdlen,exreps,cdreps,excov,cdcov,hasCDS
id,,,,,,,,,,,,,,,
CHS.1.1,known_refseq,misc_RNA,known_refseq,noncoding,yes,chr1,+,3,1652,0,.,.,0.0,.,NoCDS
CHS.2.2,known_refseq,misc_RNA,novel,undetermined,yes,chr1,-,11,1800,0,Simple_repeat:84,.,4.7,.,NoCDS
CHS.2.3,known_refseq,misc_RNA,novel,undetermined,yes,chr1,-,11,2083,0,Simple_repeat:84,.,4.0,.,NoCDS
CHS.2.4,known_refseq,misc_RNA,novel,undetermined,yes,chr1,-,11,1715,0,Simple_repeat:84,.,4.9,.,NoCDS
CHS.2.1,known_refseq,misc_RNA,known_refseq,noncoding,yes,chr1,-,11,1769,0,Simple_repeat:84,.,4.7,.,NoCDS


In [101]:
tdf['excov']=pd.to_numeric(tdf['excov'], errors='coerce').fillna(0)
tdf['cdcov']=pd.to_numeric(tdf['cdcov'], errors='coerce').fillna(0)

In [103]:
seltype=wgt.SelectMultiple(
    options=sorted(tdf.type.unique()),
    value=[],
    rows=4,
    description='TYPE',
)
selstatus=wgt.SelectMultiple(
    options=sorted(tdf.status.unique()),
    value=['novel'],
    rows=4,
    description='STATUS',
)
hbox2=wgt.HBox([selstatus, seltype])
display(hbox2)
layw=wgt.Layout(width='16em')
excov=wgt.IntSlider(value=0, min=0, max=100, step=5, layout=layw)
cdcov=wgt.IntSlider(value=0, min=0, max=100, step=5, layout=layw)
excount=wgt.BoundedIntText(value=1, min=1, max=10, description='At least',
                          layout=wgt.Layout(width='11em'))
hbox=wgt.HBox([wgt.VBox([wgt.Label(value="Min. exon cov%:"), excov]),
                  wgt.VBox([wgt.Label(value="Min. CDS cov%:"), cdcov]),
              excount, wgt.Label(value="exons")])
display(hbox)

runbtn=wgt.Button(
    description='Query',
    #disabled=False,
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Query',
    #icon='check'
)
rlabel=wgt.Label(value="---", layout=wgt.Layout(border='2px solid red', padding='0 12px'))
rlabelres=wgt.Label(value="---", layout=wgt.Layout(border='2px solid red', padding='0 12px'))
def btnRunClick(b):
    rlabel.value=''
    rdf=tdf
    if len(selstatus.value)==4 or len(selstatus.value)==0:
        rlabel.value='any status'
    else: 
        rlabel.value=' | '.join(selstatus.value)
        rdf=tdf[(tdf['status'].isin(selstatus.value))]
        if len(selstatus.value)==3 and not 'novel' in selstatus.value:
            rlabel.value='known'
        else:
            if len(selstatus.value)>1: rlabel.value='('+rlabel.value+')'
    stype=''
    
    if not(len(seltype.value)==4 or len(seltype.value)==0):
        stype=' | '.join(seltype.value)
        rdf=rdf[(rdf['type'].isin(seltype.value))]
        
    if rlabel.value and stype: 
        rlabel.value+=' &  '
        if len(seltype.value)>1: stype='('+stype+')'
    rlabel.value+=stype
    if excount.value>1:
        rdf=rdf[(rdf['excount']>=excount.value)]
    if excov.value>0:
       rdf=rdf[(rdf['excov']>=excov.value)]    
    if cdcov.value>0:
       rdf=rdf[(rdf['cdcov']>=cdcov.value)]

    rlabelres.value=str(len(rdf))

runbtn.on_click(btnRunClick)

hbx=wgt.HBox(children=[runbtn, rlabel, rlabelres], layout=wgt.Layout(justify_content='center'))
display(hbx)
btnRunClick(True)

In [104]:
cdf=pd.crosstab(tdf['status'],tdf['type'], margins=True)
del cdf.index.name
del cdf.columns.name
cdf

,coding,coding_maybe,noncoding,undetermined,All
known_fantom,0,19250,0,10791,30041
known_gencode,13272,0,36095,0,49367
known_refseq,126579,85,29727,1352,157743
novel,0,1807,0,84869,86676
All,139851,21142,65822,97012,323827


In [98]:
pd.crosstab(tdf['status'],tdf['hasCDS'], margins=True)

hasCDS,CDS,No CDS,All
status,,,
known_fantom,101,29940,30041
known_gencode,15963,33404,49367
known_refseq,114094,43649,157743
novel,1305,85371,86676
All,131463,192364,323827


In [99]:
ndf=tdf[(tdf['status']=='novel')]
print("novel transcripts (total=",ndf.shape[0],")")
nhist=go.Histogram(x=ndf['excov'], name='novel', xbins=dict(start=0.00, end=100.00, size=10.00))
data=[nhist]
layout=go.Layout(title='Repeat overlap coverage% (exons)', 
                 xaxis=dict(title='%coverage'), yaxis=dict(title='count'))
fig=go.Figure(data=data, layout=layout)
py.iplot(fig, filename='histogram')


novel transcripts (total= 86676 )
